# **FOLDER IMPLEMANTATION**

In [90]:
import numpy as np
import os
import cv2  # OpenCV for image processing
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Function to train the perceptron
def trainPerceptron(inputs, t, weights, rho, iterNo):
    # Shuffle the inputs and labels
    inputs, t = shuffle(inputs, t)

    # Add bias to inputs
    bias = np.ones((inputs.shape[0], 1))  # Create a bias term
    inputs = np.hstack((bias, inputs))  # Add bias to the input vectors

    # Training the perceptron
    for _ in range(iterNo):
        for i in range(inputs.shape[0]):
            # Compute the output (activation)
            activation = np.dot(inputs[i], weights)
            # Activation function: step function
            prediction = 1 if activation >= 0 else 0

            # Update weights if prediction does not match the target
            if prediction != t[i]:
                weights += rho * (t[i] - prediction) * inputs[i]

    # Save the trained weights
    np.save('weights.npy', weights)
    return weights

# Function to test the perceptron
def testPerceptron(sample_test, weights):
    # Load weights
    weights = np.load('weights.npy')

    # Add bias to the test sample
    bias = np.array([1])  # Create a bias term
    sample_test = np.hstack((bias, sample_test))  # Add bias to the test vector

    # Compute the output (activation)
    activation = np.dot(sample_test, weights)
    # Activation function: step function
    prediction = 1 if activation >= 0 else 0

    return prediction

# Load training data
def load_data(train_dir):
    images = []
    labels = []

    for class_folder in os.listdir(train_dir):
        class_path = os.path.join(train_dir, class_folder)
        for image_file in os.listdir(class_path):
            img = cv2.imread(os.path.join(class_path, image_file))
            if img is not None:
                img = cv2.resize(img, (224, 224))  # Resize image as necessary
                images.append(img.flatten())  # Flatten the image into a vector
                labels.append(class_folder)  # Use folder name as label

    # Convert to numpy arrays
    inputs = np.array(images)
    labels = np.array(labels)

    # Encode labels
    label_encoder = LabelEncoder()
    t = label_encoder.fit_transform(labels)  # Convert class names to integers

    return inputs, t, label_encoder

# Define paths
train_dir = '/content/train'
test_dir = '/content/test'

# Load training data
inputs, t, label_encoder = load_data(train_dir)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(inputs, t, test_size=0.2, random_state=42)

# Initialize weights randomly
weights = np.random.rand(X_train.shape[1] + 1)  # +1 for bias

# Define learning rate and iterations
rho = 0.1
iterNo = 1000

# Train the perceptron
weights = trainPerceptron(X_train, y_train, weights, rho, iterNo)

# Load testing data
test_images = []
true_test_labels = []

for class_folder in os.listdir(test_dir):
    class_path = os.path.join(test_dir, class_folder)
    for image_file in os.listdir(class_path):
        img = cv2.imread(os.path.join(class_path, image_file))
        if img is not None:
            img = cv2.resize(img, (224, 224))  # Resize image as necessary
            test_images.append(img.flatten())  # Flatten the image into a vector
            true_test_labels.append(class_folder)  # Use folder name as label

# Convert test images to numpy array
sample_tests = np.array(test_images)

# Test the perceptron on test data and calculate accuracy
correct_predictions = 0
for sample_test, true_label in zip(sample_tests, true_test_labels):
    prediction = testPerceptron(sample_test, weights)
    predicted_class = label_encoder.inverse_transform([prediction])[0]


    if predicted_class == true_label:
        correct_predictions += 1

# Calculate accuracy
accuracy = correct_predictions / len(sample_tests)
print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 42.11%


# **MORE ACCURACY**

In [92]:
import numpy as np
import os
import cv2  # OpenCV for image processing
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

# Veri yükleme fonksiyonu
def load_data(data_dir):
    images = []
    labels = []

    for class_folder in os.listdir(data_dir):
        class_path = os.path.join(data_dir, class_folder)
        if not os.path.isdir(class_path):
            continue
        for image_file in os.listdir(class_path):
            image_path = os.path.join(class_path, image_file)
            img = cv2.imread(image_path)
            if img is not None:
                img = cv2.resize(img, (64, 64))  # Boyutu küçültüyoruz
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Gri tonlamaya çeviriyoruz
                images.append(img.flatten())  # Görüntüyü vektöre dönüştürüyoruz
                labels.append(class_folder)  # Klasör ismini etiket olarak kullanıyoruz

    # NumPy dizilerine dönüştürüyoruz
    inputs = np.array(images, dtype=np.float32)
    labels = np.array(labels)

    return inputs, labels

# Aktivasyon fonksiyonu (Softmax)
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Sayısal kararlılık için
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

# Kayıp fonksiyonu (Cross-Entropy)
def cross_entropy(y_pred, y_true):
    m = y_true.shape[0]
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    loss = -np.sum(y_true * np.log(y_pred)) / m
    return loss

# Tahmin fonksiyonu
def predict(X, weights):
    z = np.dot(X, weights)
    y_pred = softmax(z)
    return y_pred

# Eğitim fonksiyonu
def train_perceptron(X_train, y_train, num_classes, learning_rate=0.01, epochs=2000):
    num_samples, num_features = X_train.shape
    weights = np.random.randn(num_features, num_classes) * 0.01

    loss_history = []

    for epoch in range(epochs):
        z = np.dot(X_train, weights)
        y_pred = softmax(z)
        loss = cross_entropy(y_pred, y_train)
        loss_history.append(loss)

        error = y_pred - y_train
        gradient = np.dot(X_train.T, error) / num_samples
        weights -= learning_rate * gradient

        if (epoch + 1) % 100 == 0:
            print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}')

    return weights, loss_history

# Veri yolları
train_dir = '/content/train'
test_dir = '/content/test'

# Eğitim verilerini yüklüyoruz
inputs, labels = load_data(train_dir)

# Etiketleri encode ediyoruz
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)

# One-hot encoding
one_hot_encoder = OneHotEncoder(sparse_output=False)
labels_one_hot = one_hot_encoder.fit_transform(labels_encoded.reshape(-1, 1))

# Verileri karıştırıyoruz
inputs, labels_one_hot = shuffle(inputs, labels_one_hot, random_state=42)

# Veriyi eğitim ve doğrulama setlerine bölüyoruz
X_train, X_val, y_train, y_val = train_test_split(inputs, labels_one_hot, test_size=0.2, random_state=42)

# Özellikleri ölçeklendiriyoruz
X_train = X_train / 255.0
X_val = X_val / 255.0

# Bias terimini ekliyoruz
X_train = np.hstack([np.ones((X_train.shape[0], 1)), X_train])
X_val = np.hstack([np.ones((X_val.shape[0], 1)), X_val])

# Modeli eğitiyoruz
weights, loss_history = train_perceptron(X_train, y_train, num_classes, learning_rate=0.001, epochs=2000)

# Ağırlıkları kaydediyoruz
np.save('weights2.npy', weights)

# Ağırlıkları yüklüyoruz
weights = np.load('weights2.npy')

# Test verilerini yüklüyoruz
test_inputs, test_labels = load_data(test_dir)
test_inputs = test_inputs / 255.0  # Özellikleri ölçeklendiriyoruz
test_inputs = np.hstack([np.ones((test_inputs.shape[0], 1)), test_inputs])  # Bias terimini ekliyoruz

# Tahmin yapıyoruz
y_pred = predict(test_inputs, weights)
predicted_classes = label_encoder.inverse_transform(np.argmax(y_pred, axis=1))

# Gerçek etiketleri alıyoruz
true_classes = test_labels

# Doğruluğu hesaplıyoruz
correct_predictions = 0
for predicted_class, true_class in zip(predicted_classes, true_classes):

    if predicted_class == true_class:
        correct_predictions += 1

accuracy = correct_predictions / len(test_inputs)
print(f'Accuracy: {accuracy * 100:.2f}%')


Epoch 100/2000, Loss: 0.8402
Epoch 200/2000, Loss: 0.7554
Epoch 300/2000, Loss: 0.6982
Epoch 400/2000, Loss: 0.6520
Epoch 500/2000, Loss: 0.6123
Epoch 600/2000, Loss: 0.5771
Epoch 700/2000, Loss: 0.5456
Epoch 800/2000, Loss: 0.5171
Epoch 900/2000, Loss: 0.4912
Epoch 1000/2000, Loss: 0.4675
Epoch 1100/2000, Loss: 0.4457
Epoch 1200/2000, Loss: 0.4256
Epoch 1300/2000, Loss: 0.4071
Epoch 1400/2000, Loss: 0.3899
Epoch 1500/2000, Loss: 0.3740
Epoch 1600/2000, Loss: 0.3591
Epoch 1700/2000, Loss: 0.3453
Epoch 1800/2000, Loss: 0.3323
Epoch 1900/2000, Loss: 0.3202
Epoch 2000/2000, Loss: 0.3089
Accuracy: 57.89%
